In [ ]:
import os
import numpy as np
import pickle5 as pickle
import re

In [ ]:
tasks=os.listdir('Training/Tasks')
tasks=tasks[1:-1]


running=True
i=0
while ((running==True) and (i<=len(tasks))):
    task=tasks[0]
    running=os.path.exists("Training/Tasks/Running/"+task)
    i=i+1
with open('Training/Tasks/Running/'+task, 'wb') as handle:
    pickle.dump(0, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [ ]:
if i>len(tasks):
    print ('all done . . . ')
else: 
    print ('Continue . . .')

In [ ]:
with open ('Training/Tasks/'+task , 'rb') as file:
    model_version=pickle.load(file)

In [3]:
import tensorflow as tf
#import tensorflow.compat.v1 as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
             tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 

Physical devices cannot be modified after being initialized


In [1]:
import chars2vec
#with tf.device('/gpu:0'):
import random
chars2vec_model = chars2vec.load_model('eng_200')
from sentence_transformers import SentenceTransformer
bert_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
import sqlite3


import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import json
import string
import itertools
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from datetime import datetime
from matplotlib import pyplot as plt
#with tf.device('/gpu:0'):
db_name = '../../../data/Bib2Auth/data1.db'
global conn
conn = sqlite3.connect(db_name,check_same_thread=False) 

In [11]:
a=[[5,4,6],[1,2,3]]
tf.keras.utils.normalize(a, axis = 1)

array([[0.56980288, 0.45584231, 0.68376346],
       [0.26726124, 0.53452248, 0.80178373]])

In [14]:
np.mean((a[0],a[1]), axis=0) 

array([3. , 3. , 4.5])

In [ ]:
# Load train, test and validation daataset
#with tf.device('/gpu:0'):
with open('Training/Miscs/new_authors_ids_'+str(model_version)+'.pickle','rb') as file:
    new_authors_ids=pickle.load(file)
with open('Training/Miscs/old_authors_ids_'+str(model_version)+'.pickle','rb') as file:
    old_authors_ids=pickle.load(file)
with open('Training/Data/x_train_new_'+str(model_version)+'.pickle','rb') as file:
    x_train=pickle.load(file)
with open('Training/Data/y_train_new_'+str(model_version)+'.pickle','rb') as file:
    y_train=pickle.load(file)
with open('Training/Data/x_validate_new_'+str(model_version)+'.pickle','rb') as file:
    x_validate=pickle.load(file)
with open('../../../data/Bib2Auth/auth2index.pickle','rb') as file:
    auth2index=pickle.load(file)

In [ ]:
old_authors_ids=list(old_authors_ids)
new_authors_ids=list(new_authors_ids)

In [ ]:



def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

try:
    with open('Training/Miscs/records_train_'+str(model_version)+'.pickle', 'wb') as handle:
        records_train=pickle.load(handle)
except:
    conn.row_factory = dict_factory
    query = "SELECT * FROM reference WHERE combination_id in "+str(tuple(x_train))
    cursor = conn.cursor()
    cursor.execute(query)
    records_train = cursor.fetchall()
    cursor.close()
    with open('Training/Miscs/records_train_'+str(model_version)+'.pickle', 'wb') as handle:
        pickle.dump(records_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
try:
    with open('Training/Miscs/records_validate_'+str(model_version)+'.pickle', 'wb') as handle:
        records_validate=pickle.load(handle)
except:
    conn.row_factory = dict_factory
    query = "SELECT * FROM reference WHERE combination_id in "+str(tuple(x_validate))
    cursor = conn.cursor()
    cursor.execute(query)
    records_validate = cursor.fetchall()
    cursor.close()
    if (conn):
        conn.close()
    with open('Training/Miscs/records_validate_'+str(model_version)+'.pickle', 'wb') as handle:
        pickle.dump(records_validate, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def generate_name_variants(auth_fname,auth_lname,coauth_fname,coauth_lname):

    # Remove all the punctuations and convert to lowercase
    auth_fname = auth_fname.translate(str.maketrans('', '', string.punctuation)).casefold()
    auth_lname = auth_lname.translate(str.maketrans('', '', string.punctuation)).casefold()
    coauth_fname = coauth_fname.translate(str.maketrans('', '', string.punctuation)).casefold()
    coauth_lname = coauth_lname.translate(str.maketrans('', '', string.punctuation)).casefold()

    auth_name = auth_fname + " " + auth_lname
    coauth_name = coauth_fname + " " + coauth_lname

    combo_list = []
    auth_name_variants = []
    coauth_name_variants = []
    
    
    
    #auth_name_variants.append(auth_name)
    #coauth_name_variants.append(coauth_name)
    
    # Check if author last name is missing and generate varinats accordingly
    if(len(auth_lname) == 0):
        auth_name_variants.append(auth_name)
        auth_name_variants.append(auth_name[0])
    else:
        auth_name_variants.append(auth_name)
        auth_name_variants.append(auth_fname[0]+ ' ' + auth_lname)
        #auth_name_variants.append(auth_fname + ' ' + auth_lname[0])

    # Check if co-author last name is missing and generate varinats accordingly
    if(len(coauth_lname) == 0):
        coauth_name_variants.append(coauth_name)
        coauth_name_variants.append(coauth_name[0])
    else:
        coauth_name_variants.append(coauth_name)
        coauth_name_variants.append(coauth_fname[0]+ ' ' + coauth_lname)
        #coauth_name_variants.append(coauth_fname + ' ' + coauth_lname[0])
 
    # Check if author and co-author are same and generate combos accordingly
    if(auth_name == coauth_name):
        combo_list = [(name,name) for name in auth_name_variants]
    else:
        combo_list = list(itertools.product(auth_name_variants,coauth_name_variants))
    return combo_list  

In [ ]:
def get_emb(combo):
    author_name = combo[0]
    
    try:
        with open('Training/tdata/auth_data/'+author_name+'.pickle','rb') as file:
            auth_emb=pickle.load(file)
    except:
        auth_emb = chars2vec_model.vectorize_words([author_name])[0]
        with open('Training/tdata/auth_data/'+author_name+'.pickle','wb') as file:
            pickle.dump(auth_emb,file)

    return auth_emb
    
def get_emb2(combo,record):
    co_author_name = combo[1]
    title = record[7]

    # take journal full form else abbr journal if full form not avaialable
    journal = record[10] if len(record[10])>0 else record[9]

    # Tranform each attribute to corresponding embeddings
    #s=time.time()
    
    
    try:
        with open('Training/tdata/auth_data/'+co_author_name+'.pickle','rb') as file:
            coauth_emb=pickle.load(file)
    except:
        coauth_emb = chars2vec_model.vectorize_words([co_author_name])[0]
        with open('Training/tdata/auth_data/'+co_author_name+'.pickle','wb') as file:
            pickle.dump(coauth_emb,file)
    
    
    try:
        with open('Training/tdata/ref_data/title'+str(record[1])+'.pickle','rb') as file:
            title_emb=pickle.load(file)
    except:
        title_emb = bert_model.encode(title)
        with open('Training/tdata/ref_data/title'+str(record[1])+'.pickle','wb') as file:
            pickle.dump(title_emb,file)
            
        
    try:
        with open('Training/tdata/ref_data/source'+str(record[1])+'.pickle','rb') as file:
                journal_emb=pickle.load(file)
    except:
        journal_emb = bert_model.encode(journal)
        with open('Training/tdata/ref_data/source'+str(record[1])+'.pickle','wb') as file:
            pickle.dump(journal_emb,file)
    
    
    
    
    
    content_emb=np.sum((title_emb,journal_emb), axis=0)               #NEW LINE
    #return np.concatenate([coauth_emb,title_emb,journal_emb])
    #return np.concatenate([auth_emb,coauth_emb,content_emb])
    return np.concatenate([coauth_emb,content_emb])
    #return auth_emb

def generate_embeddings2(record):   
    embeddings_list = []
    embeddings_list2 = []
    labels_list = []
    
    #there were a mistake in the obtention of fname and lname
    a1=record[2].split()
    a1_fname=' '.join(a1[:-1])
    a1_lname=a1[-1]
    if not bool(a1_fname):
        a1_fname=a1_lname
    a2=record[5]+' '+record[6]
    a2=a2.split()
    a2_fname=' '.join(a2[:-1])
    a2_lname=a2[-1]
    if not bool(a2_fname):
        a2_fname=a2_lname
    
    
    
    # Get the combinations of name variants
    combo_list = generate_name_variants(a1_fname,a1_lname,a2_fname,a2_lname)

    # Get the target author index
    target_author_name = record[2]
    tmp=auth2index[target_author_name]
    target_author_ind=new_authors_ids[old_authors_ids.index(tmp)]
    #target_author_ind = auth2index[target_author_name]
    
    tmp=[(embeddings_list.append(get_emb(combo)),embeddings_list2.append(get_emb2(combo,record))) for combo in combo_list]
    labels_list=[target_author_ind]*len(embeddings_list)

    embeddings_list = np.array(embeddings_list)
    embeddings_list2 = np.array(embeddings_list2)
    labels_list = np.array(labels_list)
    embeddings_list = tf.keras.utils.normalize(embeddings_list, axis = 1)
    embeddings_list2 = tf.keras.utils.normalize(embeddings_list2, axis = 1)

    return embeddings_list,embeddings_list2,labels_list

In [ ]:
def read_records_at(feature_id,t):
    if t==1:
        record=tuple(records_train[feature_id].values())
    elif t==2:
        record=tuple(records_validate[feature_id].values())
    return generate_embeddings2(record)

In [ ]:
import time 
start=time.time()

feature_id=25
record=tuple(records_train[feature_id].values())
#tmp=[i for i in range (len(ref_train)) if ref_train[i]==record[2]]
#feature_id2=random.choice(tmp)
#record2=tuple(records_train[feature_id2].values())
#feature_id3=random.choice(tmp)
#record3=tuple(records_train[feature_id3].values())

tmp=np.where(ref_train==auth2index[record[2]])
feature_id2=random.choice(tmp[0])
record2=tuple(records_train[feature_id2].values())
feature_id3=random.choice(tmp[0])
record3=tuple(records_train[feature_id3].values())
    
print (time.time()-start)

In [ ]:
records_train

In [ ]:
os.makedirs("Training/tdata/"+str(model_version)+'/train', exist_ok=True)
os.makedirs("Training/tdata/"+str(model_version)+'/validate', exist_ok=True)

def train_data_generator():
    xx=list(range(len(x_train)))
    random.shuffle(xx)
    for feature_id in xx:
        
        try:
            with open('Training/tdata/'+str(model_version)+'/train/X_'+str(feature_id)+'.pickle','rb') as file:
                X=pickle.load(file)
            with open('Training/tdata/'+str(model_version)+'/train/X2_'+str(feature_id)+'.pickle','rb') as file:
                X2=pickle.load(file)
            with open('Training/tdata/'+str(model_version)+'/train/Y_'+str(feature_id)+'.pickle','rb') as file:
                Y=pickle.load(file)
        except: 
            X,X2,Y = read_records_at(feature_id,t=1)
            with open('Training/tdata/'+str(model_version)+'/train/X_'+str(feature_id)+'.pickle','wb') as file:
                pickle.dump(X,file)
            with open('Training/tdata/'+str(model_version)+'/train/X2_'+str(feature_id)+'.pickle','wb') as file:
                pickle.dump(X2,file)
            with open('Training/tdata/'+str(model_version)+'/train/Y_'+str(feature_id)+'.pickle','wb') as file:
                pickle.dump(Y,file)
    
        for x,x2,y in zip(X,X2,Y):
            yield {"input_1": x, "input_2": x2},y
            #yield x,y

In [ ]:
def validation_data_generator():

    for feature_id in range(len(x_validate)):
        
        try:
            with open('Training/tdata/'+str(model_version)+'/validate/X_'+str(feature_id)+'.pickle','rb') as file:
                X=pickle.load(file)
            with open('Training/tdata/'+str(model_version)+'/validate/X2_'+str(feature_id)+'.pickle','rb') as file:
                X2=pickle.load(file)
            with open('Training/tdata/'+str(model_version)+'/validate/Y_'+str(feature_id)+'.pickle','rb') as file:
                Y=pickle.load(file)
        except: 
            X,X2,Y = read_records_at(feature_id,t=2)
            with open('Training/tdata/'+str(model_version)+'/validate/X_'+str(feature_id)+'.pickle','wb') as file:
                pickle.dump(X,file)
            with open('Training/tdata/'+str(model_version)+'/validate/X2_'+str(feature_id)+'.pickle','wb') as file:
                pickle.dump(X2,file)
            with open('Training/tdata/'+str(model_version)+'/validate/Y_'+str(feature_id)+'.pickle','wb') as file:
                pickle.dump(Y,file)

        for x,x2,y in zip(X,X2,Y):
            yield {"input_1": x, "input_2": x2},y
            #yield x,y


In [ ]:
_EPOCHS = 500
_PATIENCE = 50
_BATCH_SIZE = 32

_STEPS_PER_EPOCH = len(x_train)*4 // _BATCH_SIZE  
_VALIDATION_STEPS = len(x_validate)*4 // _BATCH_SIZE

train_dataset = tf.data.Dataset.from_generator(train_data_generator, ({"input_1": tf.float32, "input_2": tf.float32}, tf.int64))

train_dataset = train_dataset.batch(_BATCH_SIZE)
train_dataset = train_dataset.repeat()
train_iterator = iter(train_dataset)

validate_dataset = tf.data.Dataset.from_generator(validation_data_generator, ({"input_1": tf.float32, "input_2": tf.float32}, tf.int64))
validate_dataset = validate_dataset.batch(_BATCH_SIZE)
validate_dataset = validate_dataset.repeat()
validation_iterator = iter(validate_dataset)

In [ ]:

#with strategy.scope():
with tf.device('/gpu:0'):
    try:
        model = tf.keras.models.load_model('Models/dblp_db_model_'+str(model_version)+'.h5', compile=False)
    except:
        
        inputs1 = tf.keras.Input(shape=(200,))
        inputs2 = tf.keras.Input(shape=(968,))

        x1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(inputs2)
        x2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(inputs1)
        #d1 = tf.keras.layers.Dropout(0.2)(x2)
        x3 = tf.keras.layers.Dense(64, activation=tf.nn.relu)(x2)
        #d2 = tf.keras.layers.Dropout(0.3)(x3)
        concat_layer= tf.keras.layers.Concatenate()([x1,x3])
        d2 = tf.keras.layers.Dropout(0.3)(concat_layer)
        #d3 = tf.keras.layers.BatchNormalization()(concat_layer)
        #x5 = tf.keras.layers.Dense(128, activation=tf.nn.sigmoid)(d3)
        #d1 = tf.keras.layers.Dropout(0.2)(concat_layer)
        outputs = tf.keras.layers.Dense(len(np.unique(y_train)), activation=tf.nn.softmax)(d2)
 
        
        model = tf.keras.Model(inputs={'input_1': inputs1, 'input_2':inputs2}, outputs=outputs)
    
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

    class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train), y_train)
    class_weights = dict(enumerate(class_weights))

best_model_path = 'Models/dblp_db_model_'+str(model_version)+'.h5'
es = EarlyStopping(monitor='val_loss', mode='min', patience=_PATIENCE,verbose=1)
mc = ModelCheckpoint(filepath=best_model_path, monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)

In [ ]:
from keras.utils import plot_model
import pydot
plot_model(model,to_file='demo.png',show_shapes=True)

In [ ]:
history = model.fit(train_iterator,steps_per_epoch=_STEPS_PER_EPOCH,
                epochs=_EPOCHS,validation_data=validation_iterator,
                validation_steps=_VALIDATION_STEPS,callbacks=[es, mc],class_weight=class_weights,shuffle=True)

In [ ]:
# Dump accuracy and loss metrics

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss_file_path = "Models/loss2_"+str(model_version)+".pickle"
with open(loss_file_path,'wb') as file:
    pickle.dump(loss,file)

val_loss_file_path = "Models/val_loss2_"+str(model_version)+".pickle"
with open(val_loss_file_path,'wb') as file:
    pickle.dump(val_loss,file)
    
acc_file_path = "Models/acc2_"+str(model_version)+".pickle"
with open(acc_file_path,'wb') as file:
    pickle.dump(acc,file)
    
val_acc_file_path = "Models/val_acc2_"+str(model_version)+".pickle"
with open(val_acc_file_path,'wb') as file:
    pickle.dump(val_acc,file)